\Text Summarization with Hugging Face Transformers(DistilBERT)

dataset = https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

kaggle link :- https://www.kaggle.com/datasets/crawford/20-newsgroups
                                                                            Pretrained Model= https://huggingface.co/transformers/v3.3.1/pretrained_models.html

In [1]:
pip install ktrain==0.31

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 981 kB 61.7 MB/s 
     |████████████████████████████████| 263 kB 60.7 MB/s 
     |████████████████████████████████| 2.8 MB 65.6 MB/s 
     |████████████████████████████████| 1.3 MB 48.8 MB/s 
     |████████████████████████████████| 468 kB 66.3 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 3.3 MB 52.2 MB/s 
     |████████████████████████████████| 880 kB 55.7 MB/s 
     |████████████████████████████████| 596 kB 60.9 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.0-py3-none-any.whl size=25312260 sha256=7075685683b1f24a7bd2480b587f93078598515335ac2fdc1a5bc99ac893be3e
  Stored in directory: /root/.cache/pip/wheels/da/51/15/f25a223a50ac222ac2bde2c9d52a2a6c5352c46ac7e4625b85
  

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ['CODA_DEVICE_ORDER']='PCI_BUS_ID';
os.environ['CUDA_VISIBLE_DEVICES']='0';


In [2]:
import ktrain # lightweight wrapper for the deep learning library TensorFlow Keras that helps in building, training
from ktrain import text
from sklearn.datasets import fetch_20newsgroups #Data set from scikit learn

In [5]:
categories =['alt.atheism','soc.religion.christian','comp.graphics','sci.med','rec.sport.baseball']

In [7]:
train = fetch_20newsgroups (
      subset = 'train',
      categories=categories,
      shuffle = True,
      random_state = 0
)

In [8]:
test = fetch_20newsgroups(
      subset = 'test',
      categories=categories,
      shuffle = True,
      random_state = 0
)

In [9]:
test.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [12]:
test.target

array([0, 4, 2, ..., 2, 3, 0])

In [11]:
test.target_names

['alt.atheism',
 'comp.graphics',
 'rec.sport.baseball',
 'sci.med',
 'soc.religion.christian']

In [13]:
X_train = train.data
y_train = train.target

X_test = test.data
y_test= test.target

In [14]:
len(X_train),len(X_test)

(2854, 1899)

Build ML Model with Transformer

In [15]:
model_name = 'distilbert-base-cased'

trans = text.Transformer(model_name, maxlen=512, class_names=categories)

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [16]:
train_data=trans.preprocess_train(X_train,y_train)
test_data=trans.preprocess_test(X_test,y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 291
	95percentile : 820
	99percentile : 1757


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 323
	95percentile : 894
	99percentile : 2394


In [17]:
Model = trans.get_classifier()

Downloading:   0%|          | 0.00/354M [00:00<?, ?B/s]

In [19]:
learner = ktrain.get_learner(Model,train_data=train_data, val_data=test_data,batch_size=16)

In [ ]:
#learner.lr_find(show_plot=True, max_epochs=10)

In [20]:
learner.fit_onecycle(1e-4,1)



begin training using onecycle policy with max lr of 0.0001...
179/179 [==============================] - 207s 1s/step - loss: 0.5320 - accuracy: 0.8276 - val_loss: 0.3189 - val_accuracy: 0.9005


In [21]:
learner.validate(class_names = categories)

                        precision    recall  f1-score   support

           alt.atheism       0.84      0.69      0.76       319
soc.religion.christian       0.89      0.97      0.93       389
         comp.graphics       0.99      0.96      0.98       397
               sci.med       0.95      0.90      0.92       396
    rec.sport.baseball       0.83      0.94      0.88       398

              accuracy                           0.90      1899
             macro avg       0.90      0.89      0.89      1899
          weighted avg       0.90      0.90      0.90      1899



array([[221,  13,   3,   9,  73],
       [  6, 378,   1,   2,   2],
       [  2,   6, 382,   6,   1],
       [ 14,  25,   0, 355,   2],
       [ 19,   4,   0,   1, 374]])

In [22]:
learner.view_top_losses(n=5, preproc=trans)

----------
id:1735 | loss:6.2 | true:comp.graphics | pred:rec.sport.baseball)

----------
id:197 | loss:5.67 | true:rec.sport.baseball | pred:soc.religion.christian)

----------
id:311 | loss:5.51 | true:sci.med | pred:alt.atheism)

----------
id:1425 | loss:5.44 | true:rec.sport.baseball | pred:sci.med)

----------
id:1782 | loss:5.44 | true:sci.med | pred:alt.atheism)



Prediction on New data

In [23]:
X_test[311]

"From: turpin@cs.utexas.edu (Russell Turpin)\nSubject: Meaning of atheism, agnosticism  (was: Krillean Photography)\nOrganization: CS Dept, University of Texas at Austin\nLines: 23\nNNTP-Posting-Host: saltillo.cs.utexas.edu\n\n-*----\nSci.med removed from followups.  (And I do not read any of the\nother newsgroups.)\n\n-*----\nIn article <1993Apr30.170233.12510@rosevax.rosemount.com> grante@aquarius.rosemount.com (Grant Edwards) writes:\n> As a self-proclaimed atheist my position is that I _believe_ that \n> there is no god.  I don't claim to have any proof.  I interpret\n> the agnostic position as having no beliefs about god's existence.\n\nThat's fine.  These words have multiple meanings.\n\nAs a self-proclaimed atheist, I believe that *some* conceptions\nof god are inconsistent or in conflict with fact, and I lack\nbelief in other conceptions of god merely because there is no\nreason for me to believe in these.  I usually use the word\nagnostic to mean someone who believes that the 

In [24]:
predictor = ktrain.get_predictor(learner.model,preproc=trans)

In [27]:
X = " I have a 42 yr old male friend, misdiagnosed as having osteopporosis for two years, who recently found out that his illness is the rare Gaucher's disease."

In [28]:
predictor.predict(X)

'sci.med'